# Daily Balance Analysis 

There are four tables for you to work with:  
1.  Customer Accounts (id, name): a list of our customers with their names redacted and a unique identifier
2.  Financial Accounts (id, customer_account_id, name): a list of bank accounts belonging to our customers  
    a.  customer_account_id: The ID of the customer that owns this bank account from Customer Accounts (table 1)  
    b.  id, name: Unique financial account identifier and redacted name of the account  
3.  Financials Balances (id, account_id, amount, accrual_date): a reading of the end-of-day balance per financial account  
    a.  account_id: the ID of the financial account (table 2) that the balance reading belongs to  
    b.  amount and accrual_date: The balance (in cents) reading and the date this reading was taken  
4.  Financials Transactions (id, account_id, amount, accrual_date): list of transactions in and out of each financial account  
    a.  account_id: the ID of the financial account (table 2) that the transaction reading belongs to  
    b.  amount and accrual_date: the amount (in cents) of each transaction and the date when the transaction occurred. Negative transactions are money leaving the bank account and positive transactions are money scoming in  

# Task:  

Calculate the approximate <b>total daily balance per customer</b>  

1.  The resulting table should have <b>one row per calendar day per customer</b>.  
2.  There are neither balance readings nor transactions for every calendar day per customer. This means you will need to <b>fill in missing calendar days</b> and interpolate a balance for those days (to make things simpler, we strongly recommend using the date range 2017-01-01 to 2018-09-22 for every customer).   
3.  Be careful of other possible data integrity issues.  
4.  There are a few ways to calculate the daily balance and each one may give you slightly different daily values. Don’t worry about which one is most “correct”. Instead, just pick one method (<b>make sure to use both financials_balances and financials_transactions tables</b>) and tell us how you did it.    

In [ ]:
import numpy as np
import pandas as pd
from datetime import date

## Read the data sets

In [ ]:
!ls -lrt

total 0
-rw-r--r-- 1 nbuser nbuser        0 Feb  2 01:04 README.md
-rw-r--r-- 1 nbuser nbuser    50846 Feb  2 01:04 customer_accounts.csv
-rw-r--r-- 1 nbuser nbuser   372244 Feb  2 01:04 financials_accounts.csv
-rw-r--r-- 1 nbuser nbuser 16269644 Feb  2 01:05 financials_balances.csv
-rw-r--r-- 1 nbuser nbuser 92202701 Feb  2 01:06 financials_transactions.csv
-rw-r--r-- 1 nbuser nbuser    92479 Feb  3 22:56 Solution_LiuchengWang.ipynb


In [ ]:
customer_accounts = pd.read_csv('customer_accounts.csv')
customer_accounts.head()

,id,name
0,aacc_c10cwqghq00001owfyvwq126,Customer account 1
1,aacc_c8lodc9z00001pjfyvwq126,Customer account 2
2,aacc_czj999vto000j1mo1qkb1ydg,Customer account 3
3,aacc_celx54f0r000a1qqh4lnefn9,Customer account 4
4,aacc_cnq7b2pys000a1t4h4lnefn9,Customer account 5


In [ ]:
financials_accounts = pd.read_csv('financials_accounts.csv')
financials_accounts.head()

,id,customer_account_id,name
0,finacc_ctzrz94k00051v354687jg1,aacc_c10cwqghq00001owfyvwq126,Bank account 1
1,finacc_ctzrz8vz00031v3wyouo0dk,aacc_c10cwqghq00001owfyvwq126,Bank account 2
2,finacc_ctzrz91d00041v3h6cklh2s,aacc_c10cwqghq00001owfyvwq126,Bank account 3
3,finacc_ctzrz98x00071v3k3hz7qls,aacc_c10cwqghq00001owfyvwq126,Bank account 4
4,finacc_ctzrz9aw00081v3ghgxf64s,aacc_c10cwqghq00001owfyvwq126,Bank account 5


In [ ]:
financials_balances = pd.read_csv('financials_balances.csv', parse_dates=['accrual_date'])
financials_balances.head()

,id,account_id,amount,accrual_date
0,finblce_clqsroena000b1o70b5d0240d,finacc_clqsrob7p000a1o70h4lnefn9,20000000,2018-03-27
1,finblce_clr5aflhj008l1o70d0i79s1u,finacc_clr5afi01008k1o703vlrv3it,666181756,2017-10-13
2,finblce_clt7z6u9500b41o70np8qcjsr,finacc_clr5afi01008k1o703vlrv3it,414855384,2018-03-28
3,finblce_clv20f0yn00lu1o70kn9jksah,finacc_clqsrob7p000a1o70h4lnefn9,20000000,2018-03-28
4,finblce_csb6sn3tc0003e5a8wyouo0dk,finacc_csb6smrli0002e5a8b7haliai,12571741,2018-03-30


In [ ]:
financials_balances.dtypes

id                      object
account_id              object
amount                   int64
accrual_date    datetime64[ns]
dtype: object

In [ ]:
financials_transactions = pd.read_csv('financials_transactions.csv', parse_dates=['accrual_date'])
financials_transactions.head()

,id,account_id,amount,accrual_date
0,fintrn_clqswgddo000d1o70bw046m1l,finacc_clqsrmqo800081o70ghgxf64s,-1142,2017-10-04
1,fintrn_clqswgdgq000e1o707aru17jl,finacc_clqsrmqo800081o70ghgxf64s,-1800,2017-10-02
2,fintrn_clqsz0mzq000i1o700nzc0xi4,finacc_clqsrob7p000a1o70h4lnefn9,1043933,2018-03-26
3,fintrn_clqsz0n1b000j1o701qkb1ydg,finacc_clqsrob7p000a1o70h4lnefn9,-1200,2018-03-26
4,fintrn_clqsz0n2z000k1o70zkcqk7ti,finacc_clqsrob7p000a1o70h4lnefn9,-986752,2018-03-26


In [ ]:
financials_transactions.dtypes

id                      object
account_id              object
amount                   int64
accrual_date    datetime64[ns]
dtype: object

## Understand the data

#### There are 962 customers in the data of customer_accounts:

In [ ]:
customer_accounts.shape

(962, 2)

In [ ]:
customer_accounts.nunique()

id      962
name    962
dtype: int64

#### In the financials_accounts table, 1,556 customers own 4,704 accounts.

In [ ]:
financials_accounts.shape

(4704, 3)

In [ ]:
financials_accounts.nunique()

id                     4704
customer_account_id    1556
name                    106
dtype: int64

#### The financials_balances table only has 4,828 unique accounts and 224 unique accounting dates

In [ ]:
financials_balances.shape

(193500, 4)

In [ ]:
financials_balances.nunique()

id              193500
account_id        4828
amount           64642
accrual_date       224
dtype: int64

#### On the other hand, there are 629 days between 2017-01-01 to 2018-09-22

In [ ]:
date(2018, 9, 22) - date(2017, 1, 1)

datetime.timedelta(629)

#### The financials_transactions table only has 4,177 unique accounts and 357 unique accounting dates

In [ ]:
financials_transactions.shape

(1112797, 4)

In [ ]:
financials_transactions.nunique()

id              1112797
account_id         4177
amount           276542
accrual_date        357
dtype: int64

## Generate the result step by step

#### Join customer_accounts with financials_accounts on the customer ID

In [ ]:
customer_accounts_summary = pd.merge(customer_accounts, financials_accounts, how='left',
                                     left_on=['id'], right_on=['customer_account_id'])

In [ ]:
customer_accounts_summary.shape

(2772, 5)

In [ ]:
customer_accounts_summary.head()

,id_x,name_x,id_y,customer_account_id,name_y
0,aacc_c10cwqghq00001owfyvwq126,Customer account 1,finacc_ctzrz94k00051v354687jg1,aacc_c10cwqghq00001owfyvwq126,Bank account 1
1,aacc_c10cwqghq00001owfyvwq126,Customer account 1,finacc_ctzrz8vz00031v3wyouo0dk,aacc_c10cwqghq00001owfyvwq126,Bank account 2
2,aacc_c10cwqghq00001owfyvwq126,Customer account 1,finacc_ctzrz91d00041v3h6cklh2s,aacc_c10cwqghq00001owfyvwq126,Bank account 3
3,aacc_c10cwqghq00001owfyvwq126,Customer account 1,finacc_ctzrz98x00071v3k3hz7qls,aacc_c10cwqghq00001owfyvwq126,Bank account 4
4,aacc_c10cwqghq00001owfyvwq126,Customer account 1,finacc_ctzrz9aw00081v3ghgxf64s,aacc_c10cwqghq00001owfyvwq126,Bank account 5


In [ ]:
customer_accounts_summary.nunique()

id_x                    962
name_x                  962
id_y                   2721
customer_account_id     911
name_y                  106
dtype: int64

#### Remove useless and redundant columns/rows in the customer_accounts_summary

In [ ]:
customer_accounts_summary.columns

Index(['id_x', 'name_x', 'id_y', 'customer_account_id', 'name_y'], dtype='object')

In [ ]:
customer_accounts_summary = customer_accounts_summary.drop(columns=['name_x', 'customer_account_id', 'name_y']) 

In [ ]:
customer_accounts_summary = customer_accounts_summary.rename(columns={'id_x': 'customer_account_id', 
                                                                      'id_y': 'financial_account_id'})

In [ ]:
customer_accounts_summary.head()

,customer_account_id,financial_account_id
0,aacc_c10cwqghq00001owfyvwq126,finacc_ctzrz94k00051v354687jg1
1,aacc_c10cwqghq00001owfyvwq126,finacc_ctzrz8vz00031v3wyouo0dk
2,aacc_c10cwqghq00001owfyvwq126,finacc_ctzrz91d00041v3h6cklh2s
3,aacc_c10cwqghq00001owfyvwq126,finacc_ctzrz98x00071v3k3hz7qls
4,aacc_c10cwqghq00001owfyvwq126,finacc_ctzrz9aw00081v3ghgxf64s


In [ ]:
customer_accounts_summary[customer_accounts_summary['financial_account_id'].isnull()]

,customer_account_id,financial_account_id
128,cuacc_czjwjlwqz0065708350l6man,NaN
136,cuacc_c41zhs2e6004e72x7xf4ey4p,NaN
394,cuacc_cnrxdx1iv002r18nc803tnzg,NaN
431,cuacc_cimtf4t4c009617piz1d1upu,NaN
659,cuacc_cqse3mc97000x1wzgk5z0pru,NaN
681,cuacc_crv4pg1bz00131ri7gu33gsj,NaN
698,cuacc_cwg5oeaba000b1u4b5d0240d,NaN
753,cuacc_c4vifmxzd006w1wrhq1qdt43,NaN
873,cuacc_c5vuq4mpb05mi1vbybloxh45,NaN
882,cuacc_c60eg9tcp0c6z1tlj7mabl2z,NaN


In [ ]:
print("{} out of {} customers in the customer table have no open accounts belonging to them.".format(
       customer_accounts_summary[customer_accounts_summary['financial_account_id'].isnull()].shape[0],
       customer_accounts.shape[0]))

51 out of 962 customers in the customer table have no open accounts belonging to them.


#### Remove 51 customers who have no open accounts belonging to them.

In [ ]:
inactive_customers = list(customer_accounts_summary[customer_accounts_summary['financial_account_id'].isnull()]\
                          ['customer_account_id'])
inactive_customers

['cuacc_czjwjlwqz0065708350l6man',
 'cuacc_c41zhs2e6004e72x7xf4ey4p',
 'cuacc_cnrxdx1iv002r18nc803tnzg',
 'cuacc_cimtf4t4c009617piz1d1upu',
 'cuacc_cqse3mc97000x1wzgk5z0pru',
 'cuacc_crv4pg1bz00131ri7gu33gsj',
 'cuacc_cwg5oeaba000b1u4b5d0240d',
 'cuacc_c4vifmxzd006w1wrhq1qdt43',
 'cuacc_c5vuq4mpb05mi1vbybloxh45',
 'cuacc_c60eg9tcp0c6z1tlj7mabl2z',
 'cuacc_cbs8zr5rm01jh1zg6of15zqk',
 'cuacc_ccww9n2cs02u1129a1w6ag0n',
 'cuacc_cd1n71myh00001vufyvwq126',
 'cuacc_cext0wx1003aj1tethhe7pku',
 'cuacc_cfzz4rlxa042b1t244ogwvm5',
 'cuacc_cjub67yv800061u2p3ht5r46',
 'cuacc_ckipp6xra00i31u2bjh4oyk6',
 'cuacc_cum9g8m200ft1tmtx2fz677',
 'cuacc_c8n22risa036t1xk51pmd6xe',
 'cuacc_cb1r8kaed00101xkxny3v2h3',
 'cuacc_cfid0mkvu033l1ww45mx1nah',
 'cuacc_ckj5uj89t00c41wet7pb2yoe',
 'cuacc_cp7w9i2b704411vpy1v3oafq',
 'cuacc_cuy8ip2wx002c1tkg5vr1onv',
 'cuacc_cvzpiu5ym00031u0wyouo0dk',
 'cuacc_cw222nn7800081tughgxf64s',
 'cuacc_cxs5qkyqy02dg1sl3n44vixs',
 'cuacc_cxzqv6kxj02gp1slywkjk80d',
 'cuacc_cyauyiwcc000l

In [ ]:
cust_with_acc_summary = customer_accounts_summary[~customer_accounts_summary['financial_account_id'].isnull()]

In [ ]:
cust_with_acc_summary.nunique()

customer_account_id      911
financial_account_id    2721
dtype: int64

In [ ]:
cust_with_acc_summary.shape

(2721, 2)

#### Genreate a pivot table from financials_transactions. 

In [ ]:
financials_transactions.head()

,id,account_id,amount,accrual_date
0,fintrn_clqswgddo000d1o70bw046m1l,finacc_clqsrmqo800081o70ghgxf64s,-1142,2017-10-04
1,fintrn_clqswgdgq000e1o707aru17jl,finacc_clqsrmqo800081o70ghgxf64s,-1800,2017-10-02
2,fintrn_clqsz0mzq000i1o700nzc0xi4,finacc_clqsrob7p000a1o70h4lnefn9,1043933,2018-03-26
3,fintrn_clqsz0n1b000j1o701qkb1ydg,finacc_clqsrob7p000a1o70h4lnefn9,-1200,2018-03-26
4,fintrn_clqsz0n2z000k1o70zkcqk7ti,finacc_clqsrob7p000a1o70h4lnefn9,-986752,2018-03-26


In [ ]:
daily_transactions = pd.pivot_table(financials_transactions, values='amount', index='accrual_date',
                                    columns='account_id', aggfunc='sum', fill_value=0, dropna=False)
daily_transactions.shape

(357, 4177)

In [ ]:
daily_transactions.head()

account_id,finacc_c10t4oek002n1rly31vh9nf,finacc_c10t5h3e002p1rlbh1wr6qa,finacc_c10t5u6x002r1rlc803tnzg,finacc_c12sjsl7t00501ufb86l0zk4,finacc_c1537ukq608bk1x7v3yretrc,finacc_c1537uku708bl1x7hnfmaf8w,finacc_c1537ukwy08bm1x7qnt7mm5f,finacc_c1537ukzu08bn1x7518cahb9,finacc_c1537ul3u08bo1x7gdnswkcx,finacc_c15l7ko7y03ls1zclnia68kx,...,finacc_czy4txey300023h24b7haliai,finacc_czy4txyu500043h24h6cklh2s,finacc_czy4tycl400063h24p3ht5r46,finacc_czzlhmtw203qx1uw3nxbghdy,finacc_czzlhmy5o03qy1uw9aitzmcn,finacc_czzlhmy8s03qz1uwf48byknz,finacc_czzlhmyl803r21uwbo5awuac,finacc_czzlhmyn803r31uwlvt9pilu,finacc_czzlhmypg03r41uwrmgiyvtb,finacc_czzlhmyry03r51uwspjv186m
accrual_date,,,,,,,,,,,,,,,,,,,,,
2017-09-28,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2017-09-29,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2017-10-02,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2017-10-03,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2017-10-04,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
daily_transactions.index.min()

Timestamp('2017-09-28 00:00:00')

In [ ]:
daily_transactions.index.max()

Timestamp('2018-09-21 00:00:00')

#### Re-indexing for the pivot table. So the date ranges from 2017-01-01 to 2018-09-22

In [ ]:
all_days = pd.date_range(date(2017, 1, 1), date(2018, 9, 22), freq='D')
all_days

DatetimeIndex(['2017-01-01', '2017-01-02', '2017-01-03', '2017-01-04',
               '2017-01-05', '2017-01-06', '2017-01-07', '2017-01-08',
               '2017-01-09', '2017-01-10',
               ...
               '2018-09-13', '2018-09-14', '2018-09-15', '2018-09-16',
               '2018-09-17', '2018-09-18', '2018-09-19', '2018-09-20',
               '2018-09-21', '2018-09-22'],
              dtype='datetime64[ns]', length=630, freq='D')

In [ ]:
pd.options.display.max_rows = 700
daily_transactions_all_days = daily_transactions.reindex(all_days)
daily_transactions_all_days = daily_transactions_all_days.fillna(0)
daily_transactions_all_days.shape

(630, 4177)

#### Find the first balance of each account in the table of financials_balances

In [ ]:
financials_balances = financials_balances.sort_values('accrual_date')
financials_balances.head()

,id,account_id,amount,accrual_date
123226,finblce_c5raet8ok007y1tigiwcclbo,finacc_c5r9nl02k00051ti54687jg1,0,2016-09-28
389,finblce_cu752zce206mr1ocriglk245h,finacc_cu752zb4q06mq1ocrz4itypgz,0,2017-05-31
316,finblce_ctfpn3apj1k841xcguc4wlx17,finacc_ctfpn38ej1k831xcgkhhqtqy9,0,2017-10-06
1,finblce_clr5aflhj008l1o70d0i79s1u,finacc_clr5afi01008k1o703vlrv3it,666181756,2017-10-13
13687,finblce_ctfbd85gy0ymi1xcgcelz78od,finacc_ctfbd835f0ymh1xcgc7blezpz,422407,2017-10-25


In [ ]:
financials_fist_balances = financials_balances.drop_duplicates(subset=['account_id'], keep='first')
financials_fist_balances.shape[0] == financials_balances['account_id'].nunique()

True

In [ ]:
financials_fist_balances.head()

,id,account_id,amount,accrual_date
123226,finblce_c5raet8ok007y1tigiwcclbo,finacc_c5r9nl02k00051ti54687jg1,0,2016-09-28
389,finblce_cu752zce206mr1ocriglk245h,finacc_cu752zb4q06mq1ocrz4itypgz,0,2017-05-31
316,finblce_ctfpn3apj1k841xcguc4wlx17,finacc_ctfpn38ej1k831xcgkhhqtqy9,0,2017-10-06
1,finblce_clr5aflhj008l1o70d0i79s1u,finacc_clr5afi01008k1o703vlrv3it,666181756,2017-10-13
13687,finblce_ctfbd85gy0ymi1xcgcelz78od,finacc_ctfbd835f0ymh1xcgc7blezpz,422407,2017-10-25


#### Generate the pivot table from the table of first balances with re-indexing. So the date ranges from 2017-01-01 to 2018-09-22

In [ ]:
fist_balances = pd.pivot_table(financials_fist_balances, values='amount', index='accrual_date', 
                               columns='account_id', aggfunc='sum', fill_value=0, dropna=False)
fist_balances.shape

(204, 4828)

In [ ]:
fist_balances_all_days = fist_balances.reindex(all_days)
fist_balances_all_days = fist_balances_all_days.fillna(0)
fist_balances_all_days.shape

(630, 4828)

#### Add the table of fist_balances_all_days with the table of daily_transactions_all_days

In [ ]:
daily_transactions_all_days.shape

(630, 4177)

In [ ]:
first_balance_add_transactions_all_days = fist_balances_all_days.add(daily_transactions_all_days, fill_value=0)
first_balance_add_transactions_all_days = first_balance_add_transactions_all_days.fillna(0)
first_balance_add_transactions_all_days.shape

(630, 4839)

In [ ]:
len(set(list(fist_balances_all_days.columns) + list(daily_transactions_all_days.columns)))

4839

In [ ]:
print('Two data frames have been fully merged. There are {} unique account IDs.'.format(
      first_balance_add_transactions_all_days.shape[1]))

Two data frames have been fully merged. There are 4839 unique account IDs.


#### cumsum() to generate balance per day per account ID

In [ ]:
balances_all_days = first_balance_add_transactions_all_days.cumsum()
balances_all_days.shape

(630, 4839)

In [ ]:
balances_all_days.index

DatetimeIndex(['2017-01-01', '2017-01-02', '2017-01-03', '2017-01-04',
               '2017-01-05', '2017-01-06', '2017-01-07', '2017-01-08',
               '2017-01-09', '2017-01-10',
               ...
               '2018-09-13', '2018-09-14', '2018-09-15', '2018-09-16',
               '2018-09-17', '2018-09-18', '2018-09-19', '2018-09-20',
               '2018-09-21', '2018-09-22'],
              dtype='datetime64[ns]', length=630, freq='D')

#### Some accounts have no records in the tables of transacations and balances. Add null balacnes for those accounts every day

In [ ]:
cust_with_acc_summary.nunique()

customer_account_id      911
financial_account_id    2721
dtype: int64

In [ ]:
inactive_accounts = [acct for acct in cust_with_acc_summary['financial_account_id'].unique()
                     if acct not in balances_all_days.columns]
print('There are {} inactive accounts who have no records in the tables of transacations and balances.'.format(
      len(inactive_accounts)))

There are 24 inactive accounts who have no records in the tables of transacations and balances.


In [ ]:
inactive_accounts_balance = pd.DataFrame(columns=inactive_accounts, index=all_days)
inactive_accounts_balance = inactive_accounts_balance.fillna(0)
inactive_accounts_balance.shape

(630, 24)

In [ ]:
all_balances_all_days = pd.concat([balances_all_days, inactive_accounts_balance], axis=1)
all_balances_all_days.shape

(630, 4863)

#### Calculate total daily balance per customer. The resulting table should have one row per calendar day per customer. 

In [ ]:
account_list_for_each_customer = cust_with_acc_summary.groupby('customer_account_id')['financial_account_id'].apply(list)
account_list_for_each_customer.shape

(911,)

In [ ]:
account_list_for_each_customer.head()

customer_account_id
aacc_c10cwqghq00001owfyvwq126    [finacc_ctzrz94k00051v354687jg1, finacc_ctzrz8...
aacc_c8lodc9z00001pjfyvwq126     [finacc_clqsrob7p000a1o70h4lnefn9, finacc_clqs...
aacc_celx54f0r000a1qqh4lnefn9    [finacc_ctfm45ec71be51xcg84t7zuxc, finacc_ctfm...
aacc_cnq7b2pys000a1t4h4lnefn9    [finacc_cfu1uwl50004v196m9mdqjey, finacc_cfu1u...
aacc_cnqk1nspz000e1t47aru17jl                   [finacc_ctf6bezs30q951xcgi0v67p7p]
Name: financial_account_id, dtype: object

In [ ]:
account_list_for_each_customer['aacc_c10cwqghq00001owfyvwq126']

['finacc_ctzrz94k00051v354687jg1',
 'finacc_ctzrz8vz00031v3wyouo0dk',
 'finacc_ctzrz91d00041v3h6cklh2s',
 'finacc_ctzrz98x00071v3k3hz7qls',
 'finacc_ctzrz9aw00081v3ghgxf64s',
 'finacc_clr5afi01008k1o703vlrv3it',
 'finacc_ctzrz96l00061v3p3ht5r46',
 'finacc_ctzrytn600021v3b7haliai']

In [ ]:
# An empty pandas data frame
balance_per_account_per_day = pd.DataFrame(index=all_days)

In [ ]:
for idx in account_list_for_each_customer.index:
    balance_per_account_per_day[idx] = sum(all_balances_all_days[acct] for acct in account_list_for_each_customer[idx])

In [ ]:
balance_per_account_per_day.shape

(630, 911)

In [ ]:
balance_per_account_per_day.tail()

,aacc_c10cwqghq00001owfyvwq126,aacc_c8lodc9z00001pjfyvwq126,aacc_celx54f0r000a1qqh4lnefn9,aacc_cnq7b2pys000a1t4h4lnefn9,aacc_cnqk1nspz000e1t47aru17jl,aacc_crwybmdme000h1nj94zut58v,aacc_cxow1w7qu000z1l78rn8yn8x,aacc_cyivyslze00111l79jlzbcuz,aacc_cyjhkg4ba000r1nue5prlisl,aacc_cyu5f5iow00131nu7gu33gsj,...,cuacc_czi6jbze502rd1uhjk3bsm7t,cuacc_czk2j92q503101uh02l70gk5,cuacc_czk880l8t00041zah6cklh2s,cuacc_czqkkdf5r003x1tgtlnapr9l,cuacc_czqomtem500dp1uwebhuek9d,cuacc_czstjnk88004z1tg7dnkaca1,cuacc_czt711okh00551tghlnfytj2,cuacc_cztfod8u300zh1srm1lf0bc1,cuacc_czxekwmon0007124k3hz7qls,cuacc_czzgr8z3i02ke1r4ldqwsqp8
2018-09-18,9.141664e+09,-57416600.0,1.305794e+10,449911484.0,24322329.0,-450258539.0,-19741508.0,107680896.0,23423017.0,1.590208e+09,...,12000.0,-7.446729e+07,47026095.0,158394772.0,-87416443.0,6963823.0,48654079.0,7340059.0,3.839412e+09,254805840.0
2018-09-19,9.070412e+09,-57416600.0,1.306844e+10,446524817.0,24298879.0,-450258539.0,-19750687.0,107680896.0,23415117.0,1.590208e+09,...,272188831.0,-6.890763e+07,47026095.0,158450199.0,-89166303.0,6963823.0,51416079.0,6704398.0,3.839356e+09,248689984.0
2018-09-20,9.084632e+09,-57416600.0,1.303874e+10,446524817.0,24298879.0,-450258539.0,-19750687.0,107680896.0,23415117.0,1.590208e+09,...,268398407.0,1.018453e+09,47026095.0,158425257.0,-13346243.0,6932995.0,46919579.0,6734461.0,3.834744e+09,252067477.0
2018-09-21,9.084632e+09,-57416600.0,1.303874e+10,446524817.0,24298879.0,-450258539.0,-19750687.0,107680896.0,23415117.0,1.590208e+09,...,536796814.0,1.018453e+09,47026095.0,158425257.0,-13346243.0,6932995.0,46919579.0,6734461.0,3.834744e+09,550575593.0
2018-09-22,9.084632e+09,-57416600.0,1.303874e+10,446524817.0,24298879.0,-450258539.0,-19750687.0,107680896.0,23415117.0,1.590208e+09,...,536796814.0,1.018453e+09,47026095.0,158425257.0,-13346243.0,6932995.0,46919579.0,6734461.0,3.834744e+09,550575593.0


#### Put 51 inactive customers back. Inactive customers are the ones who have no open accounts belonging to them.

In [ ]:
balance_per_inactive_account_per_day = pd.DataFrame(columns=inactive_customers, index=all_days)
balance_per_inactive_account_per_day = balance_per_inactive_account_per_day.fillna(0)
balance_per_inactive_account_per_day.shape

(630, 51)

In [ ]:
print('The final result is as follows:')
result = pd.concat([balance_per_account_per_day, balance_per_inactive_account_per_day], axis=1)
result.shape
print('630 rows represent 630 unique days from 2017-01-01 to 2018-09-22')
print('962 columns represent 962 unique customer IDs in the customer table.')

The final result is as follows:
630 rows represent 630 unique days from 2017-01-01 to 2018-09-22
962 columns represent 962 unique customer IDs in the customer table.


In [ ]:
pd.options.display.float_format = '{:,.2f}'.format
result.tail()

,aacc_c10cwqghq00001owfyvwq126,aacc_c8lodc9z00001pjfyvwq126,aacc_celx54f0r000a1qqh4lnefn9,aacc_cnq7b2pys000a1t4h4lnefn9,aacc_cnqk1nspz000e1t47aru17jl,aacc_crwybmdme000h1nj94zut58v,aacc_cxow1w7qu000z1l78rn8yn8x,aacc_cyivyslze00111l79jlzbcuz,aacc_cyjhkg4ba000r1nue5prlisl,aacc_cyu5f5iow00131nu7gu33gsj,...,cuacc_cvq5ccyna089u1xrf3zh5vlq,cuacc_cw1fghpj707ip1wscxmgtrfs,cuacc_cw3ih9ftg0a601y7saypkv83,cuacc_cx7x4tm2x0msx1y7icfk3cmv,cuacc_cxbi7hww0000u1reln6e029c,cuacc_cykrp904x03m61txjgrfjkpn,cuacc_czitzs6ls06nd1vjsw0fodic,cuacc_czuhcv8sp031n1qi6jjy38uz,cuacc_cmvg4m5m004m1uieir2acci,cuacc_cpmyigkk00vl1xhcdxg4sdt
2018-09-18,"9,141,663,572.00","-57,416,600.00","13,057,944,700.00","449,911,484.00","24,322,329.00","-450,258,539.00","-19,741,508.00","107,680,896.00","23,423,017.00","1,590,208,088.00",...,0,0,0,0,0,0,0,0,0,0
2018-09-19,"9,070,412,144.00","-57,416,600.00","13,068,444,700.00","446,524,817.00","24,298,879.00","-450,258,539.00","-19,750,687.00","107,680,896.00","23,415,117.00","1,590,208,088.00",...,0,0,0,0,0,0,0,0,0,0
2018-09-20,"9,084,632,463.00","-57,416,600.00","13,038,744,700.00","446,524,817.00","24,298,879.00","-450,258,539.00","-19,750,687.00","107,680,896.00","23,415,117.00","1,590,208,088.00",...,0,0,0,0,0,0,0,0,0,0
2018-09-21,"9,084,632,463.00","-57,416,600.00","13,038,744,700.00","446,524,817.00","24,298,879.00","-450,258,539.00","-19,750,687.00","107,680,896.00","23,415,117.00","1,590,208,088.00",...,0,0,0,0,0,0,0,0,0,0
2018-09-22,"9,084,632,463.00","-57,416,600.00","13,038,744,700.00","446,524,817.00","24,298,879.00","-450,258,539.00","-19,750,687.00","107,680,896.00","23,415,117.00","1,590,208,088.00",...,0,0,0,0,0,0,0,0,0,0


#### Save the result as CSV file and reload it.

In [ ]:
result.to_csv('result_LiuchengWang.csv', index = True)

In [ ]:
reload_result = pd.read_csv('result_LiuchengWang.csv')
reload_result.shape

(630, 963)

In [ ]:
reload_result.tail()

,Unnamed: 0,aacc_c10cwqghq00001owfyvwq126,aacc_c8lodc9z00001pjfyvwq126,aacc_celx54f0r000a1qqh4lnefn9,aacc_cnq7b2pys000a1t4h4lnefn9,aacc_cnqk1nspz000e1t47aru17jl,aacc_crwybmdme000h1nj94zut58v,aacc_cxow1w7qu000z1l78rn8yn8x,aacc_cyivyslze00111l79jlzbcuz,aacc_cyjhkg4ba000r1nue5prlisl,...,cuacc_cvq5ccyna089u1xrf3zh5vlq,cuacc_cw1fghpj707ip1wscxmgtrfs,cuacc_cw3ih9ftg0a601y7saypkv83,cuacc_cx7x4tm2x0msx1y7icfk3cmv,cuacc_cxbi7hww0000u1reln6e029c,cuacc_cykrp904x03m61txjgrfjkpn,cuacc_czitzs6ls06nd1vjsw0fodic,cuacc_czuhcv8sp031n1qi6jjy38uz,cuacc_cmvg4m5m004m1uieir2acci,cuacc_cpmyigkk00vl1xhcdxg4sdt
625,2018-09-18,"9,141,663,572.00","-57,416,600.00","13,057,944,700.00","449,911,484.00","24,322,329.00","-450,258,539.00","-19,741,508.00","107,680,896.00","23,423,017.00",...,0,0,0,0,0,0,0,0,0,0
626,2018-09-19,"9,070,412,144.00","-57,416,600.00","13,068,444,700.00","446,524,817.00","24,298,879.00","-450,258,539.00","-19,750,687.00","107,680,896.00","23,415,117.00",...,0,0,0,0,0,0,0,0,0,0
627,2018-09-20,"9,084,632,463.00","-57,416,600.00","13,038,744,700.00","446,524,817.00","24,298,879.00","-450,258,539.00","-19,750,687.00","107,680,896.00","23,415,117.00",...,0,0,0,0,0,0,0,0,0,0
628,2018-09-21,"9,084,632,463.00","-57,416,600.00","13,038,744,700.00","446,524,817.00","24,298,879.00","-450,258,539.00","-19,750,687.00","107,680,896.00","23,415,117.00",...,0,0,0,0,0,0,0,0,0,0
629,2018-09-22,"9,084,632,463.00","-57,416,600.00","13,038,744,700.00","446,524,817.00","24,298,879.00","-450,258,539.00","-19,750,687.00","107,680,896.00","23,415,117.00",...,0,0,0,0,0,0,0,0,0,0
